## Source Detection with `find_peaks`

For more general source detection cases that do not require comparison with models, `photutils` offers the `find_peaks` function. 

This function simply finds sources by identifying local maxima above a given threshold and separated by a given distance, rather than trying to fit data to a given model. Unlike the previous detection algorithms, `find_peaks` does not necessarily calculate objects' centroids. Unless the `centroid_func` argument is passed a function like `photutils.centroids.centroid_2dg` that can handle source position centroiding, `find_peaks` will return just the integer value of the peak pixel for each source.

This algorithm is particularly useful for identifying non-stellar sources or heavily distorted sources in image data.

Let's see how it does:

In [ ]:
from photutils.detection import find_peaks
from photutils.centroids import centroid_2dg

In [ ]:
sources_findpeaks = find_peaks(xdf_image.data, mask=xdf_image.mask,
                               threshold=20. * std, box_size=29,
                               centroid_func=centroid_2dg)
print(sources_findpeaks)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped),
                      norm=norm_image, cmap=cmap)
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=30, marker='o',
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')),
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('find_peaks Sources')

And a closer look:

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_findpeaks)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['y_peak'] - cutout_size), int(src['y_peak'] + cutout_size)),
           slice(int(src['x_peak'] - cutout_size), int(src['x_peak'] + cutout_size)))
    ax.imshow(xdf_image_clipped[slc], norm=norm_image)
    src_id = np.where((sources_findpeaks['x_peak'] == src['x_peak']) &
                      (sources_findpeaks['y_peak'] == src['y_peak']))[0][0]
    ax.text(2, 2, str(src_id), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

## Comparing Detection Methods

Let's compare how each of these different strategies did.

First, how many sources did each method find?

In [ ]:
print('''DAOStarFinder: {} sources
IRAFStarFinder: {} sources
find_peaks: {} sources'''.format(len(sources_dao), len(sources_iraf_match), len(sources_findpeaks)))

Next, how many of these sources match? We can answer this question by using [sets](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset) to compare the centroids of the different sources (rounding to the nearest integer).

In [ ]:
# Make lists of centroid coordinates
centroids_dao = [(x, y) for x, y in sources_dao['xcentroid', 'ycentroid']]
centroids_iraf = [(x, y) for x, y in sources_iraf_match['xcentroid', 'ycentroid']]
centroids_findpeaks = [(x, y) for x, y in sources_findpeaks['x_centroid', 'y_centroid']]

# Round those coordinates to the ones place and convert them to be sets
rounded_centroids_dao = set([(round(x, 0), round(y, 0)) for x, y in centroids_dao])
rounded_centroids_iraf = set([(round(x, 0), round(y, 0)) for x, y in centroids_iraf])
rounded_centroids_findpeaks = set([(round(x, 0), round(y, 0)) for x, y in centroids_findpeaks])

# Examine the intersections of different sets to determine which sources are shared
all_match = rounded_centroids_dao.intersection(rounded_centroids_iraf).intersection(rounded_centroids_findpeaks)
dao_iraf_match = rounded_centroids_dao.intersection(rounded_centroids_iraf)
dao_findpeaks_match = rounded_centroids_dao.intersection(rounded_centroids_findpeaks)
iraf_findpeaks_match = rounded_centroids_iraf.intersection(rounded_centroids_findpeaks)

print('''Matching sources found by:
    All methods: {}
    DAOStarFinder & IRAFStarFinder: {}
    DAOStarFinder & find_peaks: {}
    IRAFStarFinder & find_peaks: {}'''
      .format(len(all_match), len(dao_iraf_match),
              len(dao_findpeaks_match), len(iraf_findpeaks_match)))

And just for fun, let's plot these matching sources. (The colors chosen to represent different sets are from [Paul Tol's guide for accessible color schemes](https://personal.sron.nl/~pault/).)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), norm=norm_image)
ax1.scatter([x for x, y in list(dao_findpeaks_match)], [y for x, y in list(dao_findpeaks_match)],
            s=30, marker='s', lw=1,  facecolor='None', edgecolor='#EE7733',
            label='Found by DAO and find_peaks')
ax1.scatter([x for x, y in list(dao_iraf_match)], [y for x, y in list(dao_iraf_match)],
            s=30, marker='D', lw=1, facecolor='None', edgecolor='#EE3377',
            label='Found by DAO and IRAF')
ax1.scatter([x for x, y in list(iraf_findpeaks_match)], [y for x, y in list(iraf_findpeaks_match)],
            s=30, marker='o', lw=1, facecolor='None', edgecolor='#0077BB',
            label='Found by IRAF and find_peaks')
ax1.scatter([x for x, y in list(all_match)], [y for x, y in list(all_match)],
            s=30, marker='o', lw=1.2, linestyle=':',facecolor='None', edgecolor='#BBBBBB',
            label='Found by all methods')

# Add legend
ax1.legend(ncol=2, loc='lower center', bbox_to_anchor=(0.5, -0.25))

# Define labels
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Sources Found by Different Methods')

*Remember that you can double-click on the plot to zoom in and look around!*

## Custom Detection Algorithms

If none of the algorithms we've reviewed above do exactly what you need, `photutils` also provides infrastructure for you to generate and use your own source detection algorithm: the `StarFinderBase` object can be inherited and used to develop new star-finding classes. Take a look at the [documentation](https://photutils.readthedocs.io/en/latest/api/photutils.detection.StarFinderBase.html#photutils.detection.StarFinderBase) for more information.

If you do go that route, remember that `photutils` is open-developed; you would be very welcome to [open a pull request](https://github.com/astropy/photutils/blob/master/CONTRIBUTING.rst) and incorporate your new star finder into the `photutils` source code &ndash; for everyone to use!

---

## Image Segmentation

Beyond traditional source detection methods, an additional option for identifying sources in image data is a process called **image segmentation**. This method identifies and labels contiguous (connected) objects within an image. 

You might have noticed that, in the previous source detection algorithms, large and extended sources are often incorrectly identified as more than one source. Segmentation would label all the pixels within a large galaxy as belonging to the same object, and would allow the user to then measure the photometry, centroid, and morphology of the entire object at once.

#### Creating a `SegmentationImage`

In `photutils`, image segmentation maps are created using the threshold method in the `detect_sources()` function. This method identifies all of the objects in the data that have signals above a determined **`threshold`** (usually defined as a multiple of the standard deviation) and that have more than a defined number of adjoining pixels, **`npixels`**. The data can also optionally be smoothed using a kernel, **`filter_kernel`**, before applying the threshold cut.

The `detect_sources()` function returns a `SegmentationImage` object: an array in which each object is labeled with an integer. As a simple example, a segmentation map containing two distinct sources might look like this:

```
0 0 0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0 0 0
1 1 1 1 1 0 0 0 2 0
1 1 1 1 0 0 0 2 2 2
1 1 1 0 0 0 2 2 2 2
1 1 1 1 0 0 0 2 2 0
1 1 0 0 0 0 2 2 0 0
0 1 0 0 0 0 2 0 0 0
0 0 0 0 0 0 0 0 0 0
```
where all of the pixels labeled `1` belong to the first source, all those labeled `2` belong to the second, and all null pixels are designated to be background.

Let's see what the segmentation map for our XDF data will look like.

In [ ]:
from photutils import detect_sources
from photutils.utils import make_random_cmap

In [ ]:
# Define threshold and minimum object size
threshold = 5. * std
npixels = 15

# Create a segmentation image
segm = detect_sources(xdf_image.data, threshold, npixels)

print('Found {} sources'.format(segm.max_label))

In [ ]:
# Set up the figure with subplots
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
plt.tight_layout()

# Plot the original data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped),
                      norm=norm_image, cmap=cmap)
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Original Data')

# Plot the segmentation image
rand_cmap = make_random_cmap(seed=12345)
rand_cmap.set_under('black')
segplot = ax2.imshow(np.ma.masked_where(xdf_image.mask, segm), vmin=1, cmap=rand_cmap)
ax2.set_xlabel('X (pixels)')
ax2.set_title('Segmentation Map')

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(segplot, cbar_ax)
cbar.set_label('Object Label', rotation=270, labelpad=30)

Compare the sources in original data to those in the segmentation image. Each color in the segmentation map denotes a separate source.

You can easily see that larger galaxies are shown in the segmentation map as contiguous objects of the same color - for example, the two yellow and pink galaxies near (1200, 2500). Each pixel containing light from the same galaxy has been labeled as belonging to the same object.

For a closer look, let's see what the sources we found with `find_peaks` look like in this segmentation map:

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_findpeaks)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['y_peak'] - cutout_size), int(src['y_peak'] + cutout_size)),
           slice(int(src['x_peak'] - cutout_size), int(src['x_peak'] + cutout_size)))
    ax.imshow(segm.data[slc], cmap=rand_cmap, vmin=1, vmax=len(sources_findpeaks))
    src_id = np.where((sources_findpeaks['x_peak'] == src['x_peak']) &
                      (sources_findpeaks['y_peak'] == src['y_peak']))[0][0]
    ax.text(2, 2, str(src_id), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

<div class="alert alert-block alert-info">

<h3>Exercises:</h3><br>

Recompute the <code>SegmentationImage</code>, but alter the threshold and the minimum number of pixels in a source. How does changing the threshold affect the results? What about changing the number of pixels?

</div>

#### Analyzing `source_properties`

Once we have a `SegmentationImage` object, `photutils` provides many powerful tools to manipulate and analyze the identified objects. 

Individual objects within the segmentation map can be altered using methods such as `relabel` to change the labels of objects, `remove_labels` to remove objects, or `deblend_sources` to separating overlapping sources that were incorrectly labeled as one source.

However, perhaps the most powerful aspect of the `SegmentationImage` is the ability to create a catalog using `source_properties` to measure the centroids, photometry, and morphology of the detected objects:

In [ ]:
from photutils import source_properties

In [ ]:
catalog = source_properties(xdf_image.data, segm)
table = catalog.to_table()
print(table)
print(table.colnames)

#### Creating apertures from segmentation data

We can use this information to create isophotal ellipses for each identified source. These ellipses can also later be used as photometric apertures!

In [ ]:
from photutils import EllipticalAperture

In [ ]:
# Define the approximate isophotal extent
r = 4.  # pixels

# Create the apertures
apertures = []
for obj in catalog:
    position = (obj.xcentroid.value, obj.ycentroid.value)
    a = obj.semimajor_axis_sigma.value * r
    b = obj.semiminor_axis_sigma.value * r
    theta = obj.orientation.value
    apertures.append(EllipticalAperture(position, a, b, theta=theta))

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped),
                      norm=norm_image, cmap=cmap)

# Plot the apertures
for aperture in apertures:
    aperture.plot(color='red', lw=1, alpha=0.7, axes=ax1)

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')),
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Segmentation Image Apertures')

<div class="alert alert-block alert-info">

<h3>Exercises:</h3><br>

Play with the isophotal extent of the elliptical apertures (defined above as <code>r</code>). Observe how changing this value affects the apertures that are created.

</div>

It is clear that using `photutils` for image segmentation can allow users to generate highly customized apertures &ndash; great for complex data that contain many different kinds of celestial sources.

---
## Conclusions

The `photutils` package provides users with a variety of methods for detecting sources in their data, from familar algorithms such as `DAOFind` and `starfind`, to more complex and customizable image segmentation algorithms. These methods allow for easy creation of a diverse array of apertures that can be used for photometric analysis.

**To continue with this `photutils` tutorial, go on to the [aperture photometry](../03_aperture_photometry/03_aperture_photometry.ipynb) or [PSF photometry notebook](../04_psf_photometry/04_psf_photometry.ipynb).**

---
## Additional Resources
For more examples and details, please visit the [photutils](http://photutils.readthedocs.io/en/stable/index.html) documentation.

---
## About this Notebook
**Authors:** Lauren Chambers (lchambers@stsci.edu), Erik Tollerud (etollerud@stsci.edu), Tom Wilson (towilson@stsci.edu)
<br>**Updated:** May 2019

[Top of Page](#title_ID)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/>